In [3]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 

from torchvision import datasets, transforms

In [6]:
seed = 1 # 셔플 고정

batch_size = 64
test_batch_size = 64

no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [10]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size = batch_size, shuffle = True
)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('dataset', train=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307, ), (0.3081, ))])),
batch_size = test_batch_size, shuffle = True)

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.reshape(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

In [12]:
model = Net().to(device)

In [14]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.5)


# 모델을 구성하고있는 파라미터들의 사이즈를 확인할수있음
# Weight(커널)), Bias(층을 통과 나고 여러 채널로 나올때 영향을 주는 값 같음)
params = list(model.parameters())

for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


In [16]:
# train mode로 변환
model.train()

data, target = next(iter(train_loader)) #64배치씩 하나한  가져오기
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [17]:
# 추출한 데이터를 gpu에 컴파일
data, target = data.to(device), target.to(device)

In [18]:
# gradients를 clear해서 새로운 최적화값을 찾기위해 준비
optimizer.zero_grad()

In [19]:
# 예측해보기
output = model(data)

# 모델에서 출력한 결과를 원본과비교해서 얼마나 틀렸는지 계산
loss = F.nll_loss(output, target) #Log_Likelihood Loss

# Back Propagation을 통해 Gradients(기여율)를 계산
loss.backward()

# 계산된 Gradient를 optim에 넣어줘서 optim를 업뎃해야함
optimizer.step()


In [22]:
#Hyper param 조정
epochs = 2
log_interval = 100 # 수학 log가 아닌 남는 log

In [23]:
for epoch in range(1, epochs+1):
    # train모드로 변경
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) # 뽑은 데이터를 device에 컴파일
        optimizer.zero_grad() # 기여율 초기화
        output = model(data) # 모델에 데이터 삽입하여 예측
        loss = F.nll_loss(output, target)  # 예측 결과를 target과 비교하여 오차 계산
        loss.backward() # 발생한 오차에 대해 오차의 얼만큼을 기여했는지 계산한걸로, 그만큼 lr을 곱해 weight에서 빼줌
        optimizer.step() # 옵티마이저에 변경된 weight 업뎃

        # 학습 1번 완료

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100 * batch_idx / len(train_loader), loss.item()))


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.430903
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.375004
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.519534
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.373206
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.328481
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.432391
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.312596
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.252729
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.419630
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.233712
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.350878
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.430641
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.293073
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.304014
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.271215
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.221577
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.406958
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.160343
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.291036
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.137508


# Evaluation (평가)
- 앞에서 model.train() 하여 train모드로 변경했던거 처럼, model.eval()로 설정하여 평가모드로 변경
- Batch Normalization이나 Drop Out 같은 레이어를 잠그고 실행

In [25]:
# 평가하는 행방법

model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [26]:
test_loss = 0
correct = 0

# Back Propagation이나 Gradient같은 계산을 꺼서, 속도 높아짐 // 평가땐 필요없으니깐 걍 꺼
with torch.no_grad():
    data, target = next(iter(test_loader)) # 평가용 데이터(사진, 라벨) 로드
    data, target = data.to(device), target.to(device)
    output = model(data) # 예측

    # 평가할때는 오차를 Back propagation 할 필요없음
    # 근데 일단 예측 결과와 target과의 차이 계산
    test_loss += F.nll_loss(output, target, reduction='sum').item() # reduction을 이용해 하나의 Scala로 만들어줌
     
    pred = output.argmax(dim=1, keepdim = True) # 예측 결과들(확률들) 중에 가장 큰
    correct = pred.eq(target.view_as(pred)).sum().item() # target과 pred가 같은지 체크 ==> True/False // 근데 이게 배치개수만큼 결과가 나오니깐 sum으로 True를 다 합해버려
    # 이때 view_as함수를 써서 target 얘가 pred의 행렬꼴과 같에 reshape해줌
    
    # argmax는 dim = 1이면 (가로) 64개의 이미지중 하나, 하나의 결과 10개 중 가장 큰 값의 idx를 나타냄. 총 64개를 나타내는데 이때 64개를 나타낼때, 차원을 기존 output의 차원인 2차원을 유지
    # 만약 dim = 0이면 (세로) 64개가 아닌 10개로 나타남
    
    # item()은 숫자값 반환

In [46]:
 # 7 분부터 보기

tensor([[-1.1899e+01, -9.5890e+00, -6.0238e+00, -8.8951e+00, -8.9397e+00,
         -7.1883e+00, -8.3142e+00, -8.6594e+00, -5.5478e-03, -6.4410e+00],
        [-8.5849e+00, -5.6182e-02, -5.0977e+00, -5.2562e+00, -5.5690e+00,
         -5.1753e+00, -4.9602e+00, -5.0022e+00, -4.1791e+00, -5.3807e+00],
        [-4.1546e+00, -4.0111e+00, -4.3177e+00, -2.5835e-01, -8.9893e+00,
         -1.8700e+00, -6.6589e+00, -6.0079e+00, -3.8755e+00, -6.3191e+00],
        [-1.2150e+01, -1.3669e+01, -8.2551e+00, -9.5405e-04, -1.7383e+01,
         -8.3825e+00, -1.7593e+01, -1.3933e+01, -7.7166e+00, -1.1316e+01],
        [-5.2426e+00, -1.3423e+01, -2.4755e+00, -5.5567e+00, -1.7364e+01,
         -8.7043e+00, -1.5730e+01, -1.0536e-01, -5.5563e+00, -5.9148e+00],
        [-5.3765e+00, -9.1437e+00, -3.7044e+00, -9.9278e+00, -2.0153e-01,
         -6.4189e+00, -2.2719e+00, -6.1236e+00, -4.5307e+00, -3.3404e+00],
        [-1.1147e-03, -2.5111e+01, -1.1292e+01, -1.1943e+01, -2.1941e+01,
         -6.8255e+00, -1.2733e+0